# Data Preparation - Patient Medications
This notebook extracts patient medication data from SQL Server CDWWork database and creates a unified dataset combining outpatient prescriptions (RxOut) and inpatient medication administrations (BCMA).

**Source**: SQL Server CDWWork database (RxOut + BCMA schemas)  
**Destination**: `med-data/v1_raw/medications/medications_combined.parquet`

In [1]:
# Import dependencies

import os
import sys
import logging
import time
from datetime import datetime, timedelta
import pyodbc
import boto3
import pandas as pd
import s3fs
import pyarrow as pa
import pyarrow.parquet as pq
from dotenv import load_dotenv
from importlib.metadata import version
from config import *

In [2]:
# Verify that dependencies are available for use

def print_version():
    print("boto3:", boto3.__version__)
    print("pandas:", pd.__version__)
    print("s3fs:", s3fs.__version__)
    print("pyarrow:", pa.__version__)
    print("pyodbc:", pyodbc.version)
    print("dotenv:", version("python-dotenv"))


print_version()

boto3: 1.41.5
pandas: 2.3.3
s3fs: 2025.10.0
pyarrow: 22.0.0
pyodbc: 5.3.0
dotenv: 1.2.1


In [3]:
# Set up logging

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s"
)

# Test logging
logging.info("Logging configured successfully")

2025-11-28 07:33:45,173 INFO Logging configured successfully


In [4]:
# Load configuration from config module

logging.info(f"Configuration loaded: SQL Server={SQLSERVER_SERVER}/{SQLSERVER_DATABASE}")
logging.info(f"MinIO endpoint: {MINIO_ENDPOINT}")
logging.info(f"Destination: s3://{DEST_BUCKET}/{V1_RAW_MEDICATIONS_PREFIX}")
logging.info(f"Default date range: {DEFAULT_START_DATE} to {DEFAULT_END_DATE}")
logging.info(f"BCMA action types: {BCMA_INCLUDE_ACTION_TYPES}")

2025-11-28 07:33:47,125 INFO Configuration loaded: SQL Server=127.0.0.1,1433/CDWWork
2025-11-28 07:33:47,126 INFO MinIO endpoint: localhost:9000
2025-11-28 07:33:47,127 INFO Destination: s3://med-data/v1_raw/medications/
2025-11-28 07:33:47,127 INFO Default date range: 2024-11-28 to 2025-11-28
2025-11-28 07:33:47,128 INFO BCMA action types: ['GIVEN', 'NEW BAG']


In [5]:
# Configure date range for medication extraction
# Modify these variables to adjust the date range filter

# Option 1: Use default (last 365 days)
start_date = DEFAULT_START_DATE
end_date = DEFAULT_END_DATE

# Option 2: Override with custom dates (uncomment to use)
# start_date = datetime(2024, 1, 1).date()
# end_date = datetime(2024, 12, 31).date()

logging.info(f"Using date range: {start_date} to {end_date}")
logging.info(f"Date range span: {(end_date - start_date).days} days")

2025-11-28 07:33:49,176 INFO Using date range: 2024-11-28 to 2025-11-28
2025-11-28 07:33:49,177 INFO Date range span: 365 days


In [6]:
# Create SQL Server connection

def create_sqlserver_connection():
    """
    Factory function to create SQL Server connection using pyodbc.
    Returns connection object for CDWWork database.
    """
    logging.info(f"Creating SQL Server connection to {SQLSERVER_SERVER}/{SQLSERVER_DATABASE}")
    
    conn_string = (
        f"DRIVER={{{SQLSERVER_DRIVER}}};"
        f"SERVER={SQLSERVER_SERVER};"
        f"DATABASE={SQLSERVER_DATABASE};"
        f"UID={SQLSERVER_USER};"
        f"PWD={SQLSERVER_PASSWORD};"
        f"TrustServerCertificate={SQLSERVER_TRUST_CERT};"
    )
    
    return pyodbc.connect(conn_string)


# Create the connection
conn = create_sqlserver_connection()
logging.info("SQL Server connection created successfully")
logging.info(f"Connection type: {type(conn)}")

2025-11-28 07:33:50,902 INFO Creating SQL Server connection to 127.0.0.1,1433/CDWWork
2025-11-28 07:33:50,971 INFO SQL Server connection created successfully
2025-11-28 07:33:50,971 INFO Connection type: <class 'pyodbc.Connection'>


In [7]:
# Define SQL query for unified medication dataset

# Format action types for SQL IN clause
action_types_sql = "', '".join(BCMA_INCLUDE_ACTION_TYPES)

sql_query = f"""
-- Unified medication dataset: Outpatient + Inpatient
WITH OutpatientMeds AS (
    SELECT 
        p.PatientSID,
        p.PatientIEN,
        p.Sta3n,
        p.DrugNameWithoutDose,
        p.DrugNameWithDose,
        'RxOut' AS SourceSystem,
        p.IssueDateTime AS MedicationDateTime,
        p.IssueDateTime AS StartDate,
        p.ExpirationDateTime AS EndDate,
        p.RxStatus AS Status,
        p.DaysSupply,
        p.Quantity,
        p.DEASchedule,
        p.ControlledSubstanceFlag,
        p.PrescriptionNumber AS OrderNumber,
        p.ProviderSID,
        p.LocalDrugSID,
        p.NationalDrugSID
    FROM RxOut.RxOutpat p
    WHERE p.IssueDateTime >= '{start_date}'
      AND p.IssueDateTime <= '{end_date}'
      AND p.RxStatus IN ('ACTIVE', 'DISCONTINUED', 'EXPIRED')
),
InpatientMeds AS (
    SELECT 
        m.PatientSID,
        m.PatientIEN,
        m.Sta3n,
        m.DrugNameWithoutDose,
        m.DrugNameWithDose,
        'BCMA' AS SourceSystem,
        m.ActionDateTime AS MedicationDateTime,
        m.ActionDateTime AS StartDate,
        NULL AS EndDate,
        m.ActionType AS Status,
        NULL AS DaysSupply,
        NULL AS Quantity,
        NULL AS DEASchedule,
        NULL AS ControlledSubstanceFlag,
        m.OrderNumber,
        m.OrderingProviderSID AS ProviderSID,
        m.LocalDrugSID,
        m.NationalDrugSID
    FROM BCMA.BCMAMedicationLog m
    WHERE m.ActionDateTime >= '{start_date}'
      AND m.ActionDateTime <= '{end_date}'
      AND m.ActionType IN ('{action_types_sql}')
)
SELECT * FROM OutpatientMeds
UNION ALL
SELECT * FROM InpatientMeds
ORDER BY PatientSID, MedicationDateTime;
"""

logging.info("SQL query defined")
logging.info(f"Query filters: Date range {start_date} to {end_date}")
logging.info(f"Query filters: RxOut status IN ('ACTIVE', 'DISCONTINUED', 'EXPIRED')")
logging.info(f"Query filters: BCMA action types IN {BCMA_INCLUDE_ACTION_TYPES}")

2025-11-28 07:33:53,424 INFO SQL query defined
2025-11-28 07:33:53,424 INFO Query filters: Date range 2024-11-28 to 2025-11-28
2025-11-28 07:33:53,425 INFO Query filters: RxOut status IN ('ACTIVE', 'DISCONTINUED', 'EXPIRED')
2025-11-28 07:33:53,425 INFO Query filters: BCMA action types IN ['GIVEN', 'NEW BAG']


In [8]:
# Execute query and load into DataFrame

logging.info("Executing SQL query...")
start_time = time.time()

df_medications = pd.read_sql(sql_query, conn)

elapsed = time.time() - start_time
logging.info(f"Successfully loaded {len(df_medications):,} rows, {len(df_medications.columns)} columns in {elapsed:.2f}s")

2025-11-28 07:33:57,148 INFO Executing SQL query...
/var/folders/n3/9zcf9kqj3rsb59rwpwsmy2jh0000gn/T/ipykernel_43732/3434258851.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_medications = pd.read_sql(sql_query, conn)
2025-11-28 07:33:57,163 INFO Successfully loaded 36 rows, 18 columns in 0.01s


In [9]:
# Close SQL Server connection

conn.close()
logging.info("SQL Server connection closed")

2025-11-28 07:34:00,285 INFO SQL Server connection closed


In [10]:
# Take a look at DataFrame

df_medications.head(25)

,PatientSID,PatientIEN,Sta3n,DrugNameWithoutDose,DrugNameWithDose,SourceSystem,MedicationDateTime,StartDate,EndDate,Status,DaysSupply,Quantity,DEASchedule,ControlledSubstanceFlag,OrderNumber,ProviderSID,LocalDrugSID,NationalDrugSID
0,1001,PtIEN1001,508,LISINOPRIL,LISINOPRIL 10MG TAB,BCMA,2025-01-01 08:05:00,2025-01-01 08:05:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-001001,1001,10002,20002
1,1001,PtIEN1001,508,METFORMIN HCL,METFORMIN HCL 500MG TAB,BCMA,2025-01-01 12:10:00,2025-01-01 12:10:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-001002,1001,10001,20001
2,1001,PtIEN1001,508,METFORMIN HCL,METFORMIN HCL 500MG TAB,BCMA,2025-01-01 18:08:00,2025-01-01 18:08:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-001002,1001,10001,20001
3,1001,PtIEN1001,508,LISINOPRIL,LISINOPRIL 10MG TAB,BCMA,2025-01-02 08:45:00,2025-01-02 08:45:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-001001,1001,10002,20002
4,1001,PtIEN1001,508,METFORMIN HCL,METFORMIN HCL 500MG TAB,RxOut,2025-01-15 10:30:00,2025-01-15 10:30:00,2025-01-15,ACTIVE,90.0,180.0,None,N,2024-001-0001,1001,10001,20001
5,1001,PtIEN1001,508,LISINOPRIL,LISINOPRIL 10MG TAB,RxOut,2025-01-15 10:35:00,2025-01-15 10:35:00,2025-01-15,ACTIVE,90.0,90.0,None,N,2024-001-0002,1001,10002,20002
6,1001,PtIEN1001,508,SPIRONOLACTONE,SPIRONOLACTONE 25MG TAB,RxOut,2025-03-01 10:00:00,2025-03-01 10:00:00,2025-03-01,ACTIVE,90.0,90.0,None,None,2024-001-0004,1001,10022,20022
7,1002,PtIEN1002,508,INSULIN GLARGINE,INSULIN GLARGINE 100UNIT/ML INJ,BCMA,2025-01-02 07:30:00,2025-01-02 07:30:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-002003,1002,10019,20019
8,1002,PtIEN1002,508,HYDROCODONE-ACETAMINOPHEN,HYDROCODONE-ACETAMINOPHEN 5-325MG TAB,BCMA,2025-01-02 14:35:00,2025-01-02 14:35:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-002001,1002,10012,20012
9,1002,PtIEN1002,508,ALBUTEROL SULFATE,ALBUTEROL SULFATE HFA 90MCG INHALER,RxOut,2025-01-22 09:15:00,2025-01-22 09:15:00,2025-01-22,ACTIVE,30.0,1.0,None,N,2024-002-0001,1002,10004,20004


In [11]:
# Take a look at DataFrame (tail)

df_medications.tail(25)

,PatientSID,PatientIEN,Sta3n,DrugNameWithoutDose,DrugNameWithDose,SourceSystem,MedicationDateTime,StartDate,EndDate,Status,DaysSupply,Quantity,DEASchedule,ControlledSubstanceFlag,OrderNumber,ProviderSID,LocalDrugSID,NationalDrugSID
11,1003,PtIEN1003,508,SERTRALINE HCL,SERTRALINE HCL 100MG TAB,RxOut,2025-02-05 13:20:00,2025-02-05 13:20:00,2025-02-05,ACTIVE,90.0,90.0,None,N,2024-003-0001,1001,10006,20006
12,1003,PtIEN1003,508,ALPRAZOLAM,ALPRAZOLAM 0.5MG TAB,RxOut,2025-02-05 13:25:00,2025-02-05 13:25:00,2024-08-05,DISCONTINUED,30.0,30.0,C-IV,Y,2024-003-0002,1001,10007,20007
13,1004,PtIEN1004,508,CEFTRIAXONE SODIUM,CEFTRIAXONE SODIUM 1GM IVPB,BCMA,2025-01-01 14:15:00,2025-01-01 14:15:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-004001,1002,15001,25001
14,1004,PtIEN1004,508,CEFTRIAXONE SODIUM,CEFTRIAXONE SODIUM 1GM IVPB,BCMA,2025-01-02 02:10:00,2025-01-02 02:10:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-004001,1002,15001,25001
15,1004,PtIEN1004,508,POTASSIUM CHLORIDE,POTASSIUM CHLORIDE 20MEQ IN D5W 1000ML,BCMA,2025-01-02 08:00:00,2025-01-02 08:00:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-004002,1002,15003,25003
16,1004,PtIEN1004,508,AMOXICILLIN,AMOXICILLIN 500MG CAP,RxOut,2025-03-12 15:30:00,2025-03-12 15:30:00,2024-09-12,EXPIRED,7.0,21.0,None,N,2024-004-0001,1002,10008,20008
17,1005,PtIEN1005,508,WARFARIN SODIUM,WARFARIN SODIUM 5MG TAB,BCMA,2025-01-03 17:05:00,2025-01-03 17:05:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-005001,1001,10010,20010
18,1005,PtIEN1005,508,WARFARIN SODIUM,WARFARIN SODIUM 5MG TAB,RxOut,2025-01-25 09:30:00,2025-01-25 09:30:00,2025-01-25,ACTIVE,90.0,90.0,None,N,2024-005-0001,1001,10010,20010
19,1005,PtIEN1005,508,METOPROLOL TARTRATE,METOPROLOL TARTRATE 50MG TAB,RxOut,2025-01-25 09:35:00,2025-01-25 09:35:00,2025-01-25,ACTIVE,90.0,180.0,None,N,2024-005-0002,1001,10011,20011
20,1005,PtIEN1005,508,ACETYLSALICYLIC ACID,ACETYLSALICYLIC ACID 81MG TAB,RxOut,2025-02-01 10:00:00,2025-02-01 10:00:00,2025-02-01,ACTIVE,90.0,90.0,None,None,2024-005-0003,1001,10021,20021


In [12]:
# Display DataFrame info

df_medications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   PatientSID               36 non-null     int64         
 1   PatientIEN               36 non-null     object        
 2   Sta3n                    36 non-null     int64         
 3   DrugNameWithoutDose      36 non-null     object        
 4   DrugNameWithDose         36 non-null     object        
 5   SourceSystem             36 non-null     object        
 6   MedicationDateTime       36 non-null     datetime64[ns]
 7   StartDate                36 non-null     datetime64[ns]
 8   EndDate                  19 non-null     datetime64[ns]
 9   Status                   36 non-null     object        
 10  DaysSupply               19 non-null     float64       
 11  Quantity                 19 non-null     float64       
 12  DEASchedule              2 non-null   

In [13]:
# Display DataFrame shape and memory usage

print(f"Shape: {df_medications.shape}")
print(f"Rows: {len(df_medications):,}")
print(f"Columns: {len(df_medications.columns)}")
print(f"Memory usage: {df_medications.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Shape: (36, 18)
Rows: 36
Columns: 18
Memory usage: 0.02 MB


In [14]:
# Data quality checks

print("=" * 60)
print("DATA QUALITY CHECKS")
print("=" * 60)

# Check source system distribution
print("\nSource System Distribution:")
print(df_medications['SourceSystem'].value_counts())

# Check for null drug names
null_drug_names = df_medications['DrugNameWithDose'].isnull().sum()
print(f"\nNull DrugNameWithDose: {null_drug_names} ({null_drug_names/len(df_medications)*100:.2f}%)")

# Check unique patients
unique_patients = df_medications['PatientSID'].nunique()
print(f"\nUnique patients: {unique_patients}")

# Check date range
print(f"\nDate Range:")
print(f"  Earliest: {df_medications['MedicationDateTime'].min()}")
print(f"  Latest:   {df_medications['MedicationDateTime'].max()}")

print("=" * 60)

DATA QUALITY CHECKS

Source System Distribution:
SourceSystem
RxOut    19
BCMA     17
Name: count, dtype: int64

Null DrugNameWithDose: 0 (0.00%)

Unique patients: 10

Date Range:
  Earliest: 2025-01-01 08:05:00
  Latest:   2025-04-10 14:25:00


In [15]:
# Create S3FileSystem for MinIO (pandas/pyarrow I/O)

logging.info(f"Initializing S3FileSystem for MinIO at {MINIO_ENDPOINT}")
fs = s3fs.S3FileSystem(
    anon=False,
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={
        'endpoint_url': f"http://{MINIO_ENDPOINT}"
    }
)
logging.info("S3FileSystem created successfully")

2025-11-28 07:34:19,974 INFO Initializing S3FileSystem for MinIO at localhost:9000
2025-11-28 07:34:19,976 INFO S3FileSystem created successfully


In [16]:
# Write unified medication DataFrame to v1_raw as Parquet

parquet_filename = "medications_combined.parquet"
parquet_uri = f"s3://{DEST_BUCKET}/{V1_RAW_MEDICATIONS_PREFIX}{parquet_filename}"
logging.info(f"Writing Parquet: {parquet_uri}")

start_time = time.time()

df_medications.to_parquet(
    parquet_uri,
    engine='pyarrow',
    filesystem=fs,
    compression='snappy',
    index=False
)

elapsed = time.time() - start_time
logging.info(f"Successfully wrote {len(df_medications):,} rows in {elapsed:.2f}s")

2025-11-28 07:34:21,199 INFO Writing Parquet: s3://med-data/v1_raw/medications/medications_combined.parquet
2025-11-28 07:34:21,284 INFO Successfully wrote 36 rows in 0.08s


In [17]:
# Verify write by reading back from v1_raw

logging.info("Verifying write by reading back from v1_raw...")

start_time = time.time()
df_verify = pd.read_parquet(parquet_uri, filesystem=fs)
elapsed = time.time() - start_time

logging.info(f"Verification: Read {len(df_verify):,} rows in {elapsed:.2f}s")

# Check row count matches
assert len(df_verify) == len(df_medications), f"Row count mismatch! Original: {len(df_medications)}, Read back: {len(df_verify)}"
logging.info("✓ Verification successful - row counts match")

# Check column count matches
assert len(df_verify.columns) == len(df_medications.columns), f"Column count mismatch!"
logging.info("✓ Verification successful - column counts match")

2025-11-28 07:34:23,653 INFO Verifying write by reading back from v1_raw...
2025-11-28 07:34:23,710 INFO Verification: Read 36 rows in 0.06s
2025-11-28 07:34:23,710 INFO ✓ Verification successful - row counts match
2025-11-28 07:34:23,710 INFO ✓ Verification successful - column counts match


In [18]:
# Display first few rows of verified data

df_verify.head(15)

,PatientSID,PatientIEN,Sta3n,DrugNameWithoutDose,DrugNameWithDose,SourceSystem,MedicationDateTime,StartDate,EndDate,Status,DaysSupply,Quantity,DEASchedule,ControlledSubstanceFlag,OrderNumber,ProviderSID,LocalDrugSID,NationalDrugSID
0,1001,PtIEN1001,508,LISINOPRIL,LISINOPRIL 10MG TAB,BCMA,2025-01-01 08:05:00,2025-01-01 08:05:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-001001,1001,10002,20002
1,1001,PtIEN1001,508,METFORMIN HCL,METFORMIN HCL 500MG TAB,BCMA,2025-01-01 12:10:00,2025-01-01 12:10:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-001002,1001,10001,20001
2,1001,PtIEN1001,508,METFORMIN HCL,METFORMIN HCL 500MG TAB,BCMA,2025-01-01 18:08:00,2025-01-01 18:08:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-001002,1001,10001,20001
3,1001,PtIEN1001,508,LISINOPRIL,LISINOPRIL 10MG TAB,BCMA,2025-01-02 08:45:00,2025-01-02 08:45:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-001001,1001,10002,20002
4,1001,PtIEN1001,508,METFORMIN HCL,METFORMIN HCL 500MG TAB,RxOut,2025-01-15 10:30:00,2025-01-15 10:30:00,2025-01-15,ACTIVE,90.0,180.0,None,N,2024-001-0001,1001,10001,20001
5,1001,PtIEN1001,508,LISINOPRIL,LISINOPRIL 10MG TAB,RxOut,2025-01-15 10:35:00,2025-01-15 10:35:00,2025-01-15,ACTIVE,90.0,90.0,None,N,2024-001-0002,1001,10002,20002
6,1001,PtIEN1001,508,SPIRONOLACTONE,SPIRONOLACTONE 25MG TAB,RxOut,2025-03-01 10:00:00,2025-03-01 10:00:00,2025-03-01,ACTIVE,90.0,90.0,None,None,2024-001-0004,1001,10022,20022
7,1002,PtIEN1002,508,INSULIN GLARGINE,INSULIN GLARGINE 100UNIT/ML INJ,BCMA,2025-01-02 07:30:00,2025-01-02 07:30:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-002003,1002,10019,20019
8,1002,PtIEN1002,508,HYDROCODONE-ACETAMINOPHEN,HYDROCODONE-ACETAMINOPHEN 5-325MG TAB,BCMA,2025-01-02 14:35:00,2025-01-02 14:35:00,NaT,GIVEN,NaN,NaN,None,None,IP-2025-002001,1002,10012,20012
9,1002,PtIEN1002,508,ALBUTEROL SULFATE,ALBUTEROL SULFATE HFA 90MCG INHALER,RxOut,2025-01-22 09:15:00,2025-01-22 09:15:00,2025-01-22,ACTIVE,30.0,1.0,None,N,2024-002-0001,1002,10004,20004


In [ ]:
# Summary

print("\n" + "=" * 80)
print("DATA PREPARATION SUMMARY - PATIENT MEDICATIONS")
print("=" * 80)
print(f"Source:        SQL Server {SQLSERVER_SERVER}/{SQLSERVER_DATABASE}")
print(f"Schemas:       RxOut (outpatient) + BCMA (inpatient)")
print(f"Destination:   s3://{DEST_BUCKET}/{V1_RAW_MEDICATIONS_PREFIX}{parquet_filename}")
print(f"Date Range:    {start_date} to {end_date} ({(end_date - start_date).days} days)")
print(f"Rows:          {len(df_medications):,}")
print(f"Columns:       {len(df_medications.columns)}")
print(f"Unique Pts:    {df_medications['PatientSID'].nunique()}")
print(f"RxOut Meds:    {len(df_medications[df_medications['SourceSystem']=='RxOut']):,}")
print(f"BCMA Meds:     {len(df_medications[df_medications['SourceSystem']=='BCMA']):,}")
print(f"Status:        ✓ Complete")
print("=" * 80)
print("\nNext: Run remaining 01x_dataprep notebooks")
print("Then: Run 02_explore.ipynb for exploratory data analysis")


DATA PREPARATION SUMMARY - PATIENT MEDICATIONS
Source:        SQL Server 127.0.0.1,1433/CDWWork
Schemas:       RxOut (outpatient) + BCMA (inpatient)
Destination:   s3://med-data/v1_raw/medications/medications_combined.parquet
Date Range:    2024-11-28 to 2025-11-28 (365 days)
Rows:          36
Columns:       18
Unique Pts:    10
RxOut Meds:    19
BCMA Meds:     17
Status:        ✓ Complete

Next: Run remaining 01x_dataprep notebooks
Then: Run 02_explore.ipynb for exploratory data analysis
Note: Can now join medications_combined.parquet with DDI reference data
